In [28]:
from pyspark.sql import SparkSession
import getpass
username = getpass.getuser()
spark = SparkSession. \
builder. \
config('spark.ui.port', '0'). \
config('spark.shuffle.useOldFetchProtocol','true'). \
config("spark.sql.warehouse.dir", f"/user/itv021558/warehouse"). \
enableHiveSupport(). \
master('yarn'). \
getOrCreate()

In [15]:
spark.sql("select member_id, count(*) as total from itv021558_LendingClub.customers group by member_id order by total desc")

member_id,total
e4c167053d5418230...,5
3f87585a20f702838...,4
ad8e5d384dae17e06...,4
76b577467eda5bdbc...,4
3ae415acd6bbfaac1...,3
059d401bb603d9a80...,3
d93573e2883e37904...,3
066ddaa64bee66dff...,3
a2356725aa9da92c6...,3
4ab6205de571ccb7b...,3


In [29]:
bad_data_customers_df = spark.sql("""select member_id from (select member_id, 
count(*) as total from itv021558_LendingClub.customers group by member_id having total > 1)""")

In [5]:
bad_data_customers_df.count()

3157

In [8]:
bad_data_customers_df

member_id
eebbd89aa7efc13e7...
5130d0087970e032e...
cbede54df344cdb94...
761b2f1e61999e14e...
7115ace193e13d8f3...
4231a55d0199c619a...
3b8ffe19f24749a73...
d4782ddad8591f44d...
01b2223757c3b62e7...
f284044b881f218c0...


In [30]:
bad_data_loans_defaulters_delinq = spark.sql("""select member_id from (select member_id, 
count(*) as total from itv021558_LendingClub.loans_defaulters_delinq group by member_id having total > 1)""")

In [10]:
bad_data_loans_defaulters_delinq.count()

173

In [31]:
bad_data_loans_defaulters_detail_rec_enq = spark.sql("""select member_id from (select member_id, 
count(*) as total from itv021558_LendingClub.loans_defaulters_detail_rec_enq group by member_id having total > 1)""")

In [19]:
bad_data_loans_defaulters_detail_rec_enq.count()

3189

In [20]:
bad_data_customers_df.repartition(1).write \
.format("csv") \
.option("header", True) \
.mode("overwrite") \
.option("path", "/user/itv021558/lendingclubproject/bad/bad_customers") \
.save()

In [21]:
bad_data_loans_defaulters_delinq.repartition(1).write \
.format("csv") \
.option("header", True) \
.mode("overwrite") \
.option("path", "/user/itv021558/lendingclubproject/bad/bad_data_loans_defaulters_delinq") \
.save()

In [32]:
bad_cust_data_df = bad_data_customers_df.select("member_id") \
.union(bad_data_loans_defaulters_delinq.select("member_id")) \
.union(bad_data_loans_defaulters_detail_rec_enq.select("member_id")) \

In [33]:
bad_cust_final = bad_cust_data_df.distinct()

In [25]:
bad_cust_final.count()

3189

In [26]:
bad_cust_final.repartition(1).write \
.format("csv") \
.option("header", True) \
.mode("overwrite") \
.option("path", "/user/itv021558/lendingclubproject/bad/bad_customer_data_final") \
.save()

In [34]:
bad_cust_final.createOrReplaceTempView("customers")

In [35]:
customers_df = spark.sql(""" select * from itv021558_LendingClub.customers 
where member_id NOT IN (select member_id from customers)""")

In [36]:
customers_df.count()

2254192

In [39]:
customers_df.write \
.mode("overwrite") \
.option("path", "/user/itv021558/lendingclubproject/cleaned_new/customers_parquet") \
.save()

In [40]:
loans_def_delinq_df = spark.sql(""" select * from itv021558_LendingClub.loans_defaulters_delinq 
where member_id NOT IN (select member_id from customers)""")

In [41]:
loans_def_delinq_df.write \
.mode("overwrite") \
.option("path", "/user/itv021558/lendingclubproject/cleaned_new/loans_def_delinq_df_parquet") \
.save()

In [42]:
loans_defaulters_detail_rec_enq = spark.sql(""" select * from itv021558_LendingClub.loans_defaulters_detail_rec_enq 
where member_id NOT IN (select member_id from customers)""")

In [43]:
loans_defaulters_detail_rec_enq.write \
.mode("overwrite") \
.option("path", "/user/itv021558/lendingclubproject/cleaned_new/loans_defaulters_detail_rec_enq_parquet") \
.save()

In [44]:
spark.sql("""
CREATE EXTERNAL TABLE
itv021558_LendingClub.customers_new(member_id string,emp_title string,emp_length int, home_ownership string,annual_income float,
address_state string,address_zipcode string,address_country string,grade string,sub_grade string,verification_status
string,total_high_credit_limit float,application_type string,join_annual_income float,verification_status_joint string,
ingest_date timestamp) stored as parquet
LOCATION
'/user/itv021558/lendingclubproject/cleaned_new/customers_parquet'
""")

""


In [45]:
spark.sql("""
CREATE EXTERNAL TABLE
itv021558_LendingClub.loans_defaulters_delinq_new(member_id
string,delinq_2yrs integer,delinq_amnt float,mths_since_last_delinq integer)
stored as parquet
LOCATION
'/user/itv021558/lendingclubproject/cleaned_new/loans_defaulters_delinq_parquet'
""")

""


In [46]:
spark.sql("""
CREATE EXTERNAL TABLE
itv021558_LendingClub.loans_defaulters_detail_rec_enq_new(member_id string,
pub_rec integer, pub_rec_bankruptcies integer, inq_last_6mths integer) stored
as parquet
LOCATION
'/user/itv021558/lendingclubproject/cleaned_new/loans_defaulters_detail_records_enq_parquet'
""")

""


#### Removed duplicate enteries for same member_id's

In [47]:
spark.sql("""select member_id ,count(*) as total from itv021558_LendingClub.loans_defaulters_delinq_new
group by member_id order by total desc
""")

member_id,total
